# WSW Results

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import glob
import jinja2

from datetime import datetime
import time

import csv
import json

from PIL import Image

from common import Printable, projdir

from country import countryNames
from event import Event

from constants import *

## Process Years

Process all available years

In [2]:
logger = Printable()

events = {}
existingNames = {}

def processEvents():
    '''Initialise events - load entrants and index sails / GT-31s'''

    eventPaths = sorted(glob.glob(os.path.join(projdir, EVENTS_DIR, '[1-2][0-9][0-9][0-9]')))
    currYear = datetime.today().year
    for eventPath in eventPaths:
        eventYear = int(os.path.basename(eventPath))

        if eventYear == currYear or appConfig['History']['Refresh']:
            if eventYear == currYear:
                verbosity = appConfig['Latest']['Verbosity']
            else:
                verbosity = appConfig['History']['Verbosity']

            event = Event(eventPath, appConfig, existingNames=existingNames, verbosity=verbosity)
            event.processEvent()

            events[event.year] = event


def getBanner():
    '''Get banner details'''
    
    bannerFn = appConfig['Event']['Banner']
    banner = {}

    banner['src'] = os.path.join('..', 'img', bannerFn)
    banner['alt'] = appConfig['Event']['Name']

    with Image.open(os.path.join(projdir, 'docs', 'img', bannerFn)) as bannerImage:
        banner['width'], banner['height'] = bannerImage.size

    return banner


def createIndex():
    '''Create index for available years'''

    banner = getBanner()
    
    eventName = appConfig['Event']['Name']
    pageTitle = 'Events'
    pageDescription=pageTitle

    subDirs = []
    for eventId in reversed(events):
        event = events[eventId]

        if len(event.entrants) > 0 or event.comment:
            href = os.path.join(str(eventId), 'index.html')
            eventTitle = eventId

            fastest, speed = event.getFastest()
            
            subDir = {'id': eventId, 'href': href, 'label': eventTitle, 'fastest': fastest, 'speed': speed}
            subDirs.append(subDir)

    htmlDir = os.path.join(projdir, 'docs', 'results')
    htmlFile = os.path.join(htmlDir, 'index.html')

    templatePath = os.path.join(projdir, 'python', 'templates')
    templateLoader = jinja2.FileSystemLoader(searchpath=templatePath)
    templateEnv = jinja2.Environment(loader=templateLoader, autoescape=True, trim_blocks=True, lstrip_blocks=True)

    template = templateEnv.get_template("index.html")
    html = template.render(
                    eventName=eventName,
                    banner=banner,
                    className=None,
                    pageTitle=pageTitle,
                    pageComment=None,
                    pageDescription=pageDescription,
                    periodDate=None,
                    subDirTitle = None,
                    cssPath=os.path.join('..', 'css'),
                    homePage='index.html',
                    files=[],
                    subDirs=subDirs)

    with open(htmlFile, 'w', encoding='utf-8') as f:
        f.write(html)

In [3]:
if __name__ == '__main__':
    pc1 = time.perf_counter()
    
    # Read main config
    filename = os.path.join(projdir, CONFIG_DIR, APP_CONFIG)
    with open(filename, 'r', encoding='utf-8') as f:
        jsonTxt = f.read()
        try:
            appConfig = json.loads(jsonTxt)
        except:
            logger.logError('Could not parse {}'.format(filename))
            raise
    
    processEvents()
    createIndex()
    
    pc2 = time.perf_counter()
    print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Processing 1998...
All done!

Processing 1999...
All done!

Processing 2000...
All done!

Processing 2001...
All done!

Processing 2002...
All done!

Processing 2003...
All done!

Processing 2004...
All done!

Processing 2005...
All done!

Processing 2006...
All done!

Processing 2007...
All done!

Processing 2008...
All done!

Processing 2009...
All done!

Processing 2010...
All done!

Processing 2011...
All done!

Processing 2012...
All done!

Processing 2013...
All done!

Processing 2014...
All done!

Processing 2015...
All done!

Processing 2016...
All done!

Processing 2017...
All done!

Processing 2018...
All done!

Processing 2019...
All done!

Processing 2020...
All done!

Processing 2021...
All done!

Processing 2022...
All done!

Reports completed in 10.96 seconds


## All Done!